### Chapter 1: Setup and data retrieval

As in the other notebooks, we must first setup hylite in the Colab environment and then download the data. In this case, we download (1) the hyperclouds created in Notebook 1, and (2) the minimum wavelength maps generated in Notebook 2.


### Chapter 2: Band ratios and saturation enhanced visualisation

Even simple processing methods can produce qualitatively useful visualisations of hyperspectral data. In the following we apply some well established band ratios to create a false-colour point cloud that captures variation in iron mineralogy, and a saturation enhanced false-colour image that conveys the geological variability captured in the SWIR range.

In [1]:
import hylite # if this doesn't work then please refer to Step 1
from hylite import io
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from hylite.analyse import band_ratio
from hylite.filter import boost_saturation
from hylite.correct import get_hull_corrected
from hylite.analyse import colourise_mwl, plot_ternary
from hylite.project import Camera
from hylite.project import proj_pano
import matplotlib.gridspec as gridspec

In [2]:
# figure settings for matplotlib
plt.rc('font', size=14)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=12)    # legend fontsize
plt.rc('figure', titlesize=20)  # fontsize of the figure title
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["axes.facecolor"] = "white"
mpl.rcParams["savefig.facecolor"] = "white"
c = 'black'
mpl.rcParams['text.color'] = c
mpl.rcParams['axes.labelcolor'] = c
mpl.rcParams['xtick.color'] = c
mpl.rcParams['ytick.color'] = c

In [3]:
# load hyperclouds HyCollection
C = io.load('hyperclouds.hyc')

In [4]:
# create output for storing mapping results
O = hylite.HyCollection( 'visualisations','results/')

Compute band ratios showing iron mineralogy

In [5]:
for n in ['M1', 'M4', 'N1', 'N2']:
    print("Loading data for %s" % n, end = '')
    cloud = C.get(n)
    bands = np.full(cloud.band_count(), False)
    bands[ [ cloud.get_band_index(b) for b in [570.,600., 2280., 2245.] ] ] = True # bands needed for Fe3 and FeOH ratios
    bands[ cloud.get_band_index(920.):cloud.get_band_index(1650.) ] = True # band range needed for Fe2 ratio
    cloud = cloud.export_bands(bands) # save RAM before decompressing by grabbing only bands we need
    cloud.decompress()
    cloud.set_as_nan(0)

    # filter / clean point cloud
    valid = np.isfinite(cloud.data).all(axis=-1) & (cloud.data != 0).any(axis=-1)
    cloud.filter_points(0, np.logical_not(valid))

    # iron band ratios
    print(". Generating: iron indices...", end = '')
    Fe3 = band_ratio( cloud, 600., 570. )
    Fe2 = band_ratio( cloud, [920.,1650.],[1035.,1230.])
    FeOH = band_ratio( cloud, 2280., 2245. )

    stack = cloud.copy( data = False )
    stack.data = np.hstack( [(Fe3.data - 1.0) / 0.3, (Fe2.data-0.95) / 0.1, (FeOH.data - 1.0) / 0.3 ] ) # build RGB data and apply colour stretch
    stack.colourise((0,1,2), stretch = (0.0, 1.0 ) ) # clip and push to 0-255 RGB values
    stack.data = None # we can delete scalar fields now
    O.set(n+"_Fe", stack)

    # we don't need the hypercloud anymore
    del cloud
    C.free()

    # cleanup and save
    print("Saving...", end = '')
    O.save()
    O.free()
    print("Done.")

Loading data for M1. Generating: iron indices...Saving...Done.
Loading data for M4. Generating: iron indices...Saving...Done.
Loading data for N1. Generating: iron indices...Saving...Done.
Loading data for N2. Generating: iron indices...Saving...Done.


And saturation enhanced false-colour composites

In [ ]:
for n in ['M1', 'M4', 'N1', 'N2']:
    print("Loading data for %s..." % n, end = '')
    cloud = C.get(n).export_bands((2100., 2430.)) # load hypercloud
    C.free()
    cloud.decompress()
    cloud.set_as_nan(0)

    # filter / clean point cloud
    valid = np.isfinite(cloud.data).all(axis=-1) & (cloud.data != 0).any(axis=-1)
    cloud.filter_points(0, np.logical_not(valid))

    # hull correction
    print("hull correction...", end = '')
    cloud = get_hull_corrected( cloud, band_range=(2100., 2430.) )

    # saturation enhanced composites
    print("Enhanced composite...", end = '')
    boosted = boost_saturation( cloud, hylite.SWIR, clip=(2,98), sat=0.7, val=0.85 )
    boosted.colourise((0,1,2), stretch=(0,100))
    boosted.data = None # we can delete scalar fields now
    O.set(n+"_ENH", boosted)

    # we don't need the hypercloud anymore
    del cloud

    # cleanup and save
    print("Saving...", end = '')

    O.save()
    O.free()
    print("Done.")

Loading data for M1...hull correction...

Removing hull:   0%|          | 0/5629777 [00:00<?, ?it/s]

Plot everything on a fancy figure

In [ ]:
fig,ax = plt.subplots(4,2,figsize=(20,14))
l = 'abcdefgh'
for i,n in enumerate(['N1', 'N2', 'M1', 'M4']):

    # get data
    FeO = O.get(n+"_Fe")
    enh = O.get(n+"_ENH")
    cam = C.get(n+'_cam')
    # render scenes and plot
    t = ['FeOx', 'ENH']
    for j,img in enumerate([FeO,enh]):
        # render image
        render = img.render(cam,'rgb')
        render.fill_holes()
        render.data[(render.data==0).all(axis=-1),:] = np.nan # remove zeros after fill_holes

        # clip to data area
        ymin,ymax = np.percentile( np.argwhere( np.sum( np.isfinite(render.data[...,0]), axis=0 ) != 0 ),(0,100) )
        xmin,xmax = np.percentile( np.argwhere( np.sum( np.isfinite(render.data[...,0]), axis=1 ) != 0 ),(0,100) )
        render.data = render.data[int(xmin):int(xmax), int(ymin):int(ymax), :]

        # plot
        render.quick_plot((0,1,2), ax=ax[i,j],vmin=0.,vmax=255.)
        ax[i,j].set_title(l[i*2+j] + ". Scene " + n + ' (%s)'% t[j])
        ax[i,j].set_frame_on(False)
fig.tight_layout()
fig.show()

### Chapter 3: Minimum wavelength maps

We also generate visualisations of the minimum wavelength maps. These were calculated as described in Notebook 2, and were downloaded at in the first part of this notebook.

In [ ]:
# helps keep memory usage down
%reset -f

In [ ]:
import hylite # if this doesn't work then please refer to Step 1
from hylite import io
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from hylite.analyse import colourise_mwl, plot_ternary
from hylite.project import Camera
from hylite.project import proj_pano
import matplotlib.gridspec as gridspec

In [ ]:
# figure settings for matplotlib
plt.rc('font', size=14)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=12)    # legend fontsize
plt.rc('figure', titlesize=20)  # fontsize of the figure title
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["axes.facecolor"] = "white"
mpl.rcParams["savefig.facecolor"] = "white"
c = 'black'
mpl.rcParams['text.color'] = c
mpl.rcParams['axes.labelcolor'] = c
mpl.rcParams['xtick.color'] = c
mpl.rcParams['ytick.color'] = c

Load hypercloud collection and MWL mapping results

In [ ]:
C = io.load('hyperclouds.hyc') # load hyperclouds HyCollection
M = io.load('1000mwl_maps.hdr')
#M = io.load('mwl_maps.hdr') # load MWL features
O = hylite.HyCollection( 'visualisations','results/') # create output for storing mapping results

To quickly summarise the dominant spectral features, we will create a false-colour image with the depth of the three dominant SWIR features (AlOH, FeOH and MgOH) mapped to RGB. Additionally, we will plot the distribution of feature depth and position on a modified ternary diagram to highlight general trends.

In [ ]:
fig,ax = plt.subplots(4,2,figsize=(20,25))
for i,(n,t) in enumerate(zip(['M1', 'M4', 'N1', 'N2'],'abcd')):
    print("Loading and plotting %s" % n)
    mwl = M.get(n + "_MWL") # load hypercloud
    cam = C.get(n+"_cam")

    # extract features
    F2200 = mwl.deepest(2100., 2235.)
    F2250 = mwl.deepest(2235., 2290.)
    F2350 = mwl.deepest(2290., 2380.)
    features = [F2200, F2250, F2350]

    O.free() # free memory
    M.free()

    # create dominant feature composite
    domF = F2200.copy(data=False)
    domF.data = np.vstack([F2200[:,0], F2250[:,0], F2350[:,0] / 3. ]).T # get depth of each feature
    domF.set_band_names(['F2200_depth','F2250_depth','F2350_depth'])
    domF.data = np.nan_to_num(domF.data) # replace nans with zeros
    domF.rgb = domF.data / np.sum(domF.data, axis=-1)[:,None] # convert to colours
    domF.rgb = (np.clip(domF.rgb,0,1) * 255).astype(np.uint8) # convert to uint8

    # build RGB false-colour minimum wavelength maps for each feature
    hue_map = [(2190.,2230.),(2230.,2290.),(2300.,2350.)]
    for j,(f,fn) in enumerate(zip( features, ['F2200','F2250','F2350'] )):
        # compute RGB mapping
        rgb,leg = colourise_mwl(f, strength=False, mode='pd-', hue_map = hue_map[j], val_map=(0.0,0.1))

        # bind it to the point cloud
        f.rgb = rgb.rgb
        f.rgb[ (f.rgb == 0).all(axis=-1), : ] = 204 # replace black (nans) with gray
        f.set_band_names(["depth","pos","width_L", "width_R"])

        # and store
        O.set("%s_%s"%(n,fn), f)

    # store in the results collection
    O.set(n+'_domF', domF)
    O.save()

    # compute hillshade
    klm = F2200.render(cam,'klm',fill_holes=True)
    sundir = np.mean(klm.data,axis=(0,1))
    sundir = sundir / np.linalg.norm(sundir)
    shade = np.clip( np.dot(klm.data, sundir), 0.5, 1.0 )

    # add to plot
    ax[i,0].set_title('%s. %s absorption depth'%(t,n),size=25, loc='left')


    plot_ternary( F2200, F2250, F2350, bounds=[(2160,2235),(2235,2290),(2290,2380)], weights=[1.,1.,1./3],
                    labels=['AlOH','FeOH', 'CO3'], s=10,palpha=0.05,label_offset=0.4,
                    subsample=75, depth_thresh=0.1,ax=ax[i,1],
                    title='',)

    render = domF.render(cam,'rgb') # render the cloud!
    render.fill_holes() # fill gaps in point cloud with neighbouring values
    render.despeckle(size=3) # apply moderate median filter to reduce speckle noise
    render.data[(render.data==0).all(axis=-1),:] = np.nan # remove zeros after fill_holes
    render.data*=shade[...,None] # apply hillshade
    render.crop_to_data() # clip to data area
    render.data = render.data[max(0,render.xdim()-render.ydim()*3):-1,:,:] # clip to fit in figure
    render.quick_plot((0,1,2), ax=ax[i,0],vmin=0.,vmax=255.)
fig.tight_layout()
fig.show()

The position of the absorption features also reveals important information on mineral composition, especially (in this case) for distinguishing calcitic from dolomitic marbles. It is common to convey this information by mapping feature position to hue and depth to saturation or brightness. To illustrate this, we'll plot the minimum wavelength mapping results for the main Maarmorilik scene.

In [ ]:
# create camera for plotting
pos = np.array([489430.,7888040.,50.])
ori = np.array([-100,50,-10])
dims = (1500,500)
fov=15
cam = Camera( pos, ori, 'pano', fov=fov, dims=dims, step=fov / dims[1])

In [ ]:
# build RGB false-colour minimum wavelength maps for each feature
hue_map = [(2190.,2230.),(2230.,2290.),(2300.,2350.)]
features = [O.M1_F2200,O.M1_F2250,O.M1_F2350]
rgb = [colourise_mwl(m, strength=False, mode='pd-', hue_map = hue_map[i], val_map=(0.0,0.1))
                                                                       for i,m in enumerate(features)]

In [ ]:
# compute hillshade
klm = features[0].render(cam,'klm',fill_holes=True)
sundir = np.mean(klm.data,axis=(0,1))
sundir = sundir / np.linalg.norm(sundir)
shade = np.clip( np.dot(klm.data, sundir), 0.5, 1.0 )

In [ ]:
# extract some spectra to check the MWL results
idxA = [1050666, 1052923, 1258779, 1261761]
namesA = ['Tremolitic marble', 'Dolomitic marble (E)', 'Calcitic marble (E)', 'Pelite']
idxB = [117507,  160292,  173009,  236904]
namesB = ['Calcitic marble (W)', 'Dolomitic marble (W)', 'Black Angel pelite', 'Graphitic marble']

# put results in a spectral library
A = hylite.HyLibrary( C.M1.data[idxA,:], lab=namesA, wav=C.M1.get_wavelengths() )
B = hylite.HyLibrary( C.M1.data[idxB,:], lab=namesB, wav=C.M1.get_wavelengths() )
A.decompress()
B.decompress()

# extract positions also
A.xyz = C.M1.xyz[idxA,:]
B.xyz = C.M1.xyz[idxB,:]

In [ ]:
# plot features
fig,ax = plt.subplots(4,1,figsize=(20,26))
names=['a. $AlOH$ feature', 'b. $FeOH$ feature', 'c. $CO_{3}$ / $MgOH$ feature']
for i, (cld,leg) in enumerate(rgb):
    # generate cloud
    cld.rgb[ (cld.rgb == 0).all(axis=-1), : ] = 204 # replace black (nans) with gray
    cld.data = features[i].data # copy data from MWL rather than RGB info ( for export )
    cld.set_wavelengths(None)
    cld.set_band_names(["depth","pos","width_L", "width_R"])

    # render cloud
    render = cld.render(cam,'rgb')
    render.fill_holes()
    render.despeckle(3)
    render.data[(render.data==0).all(axis=-1),:] = np.nan # remove zeros after fill_holes
    render.data *= shade[...,None] # apply hillshade

    # plot it
    ax[i].set_title(names[i],loc='left')
    render.quick_plot((0,1,2),vmin=0.,vmax=255.,ax=ax[i])
    leg.plot(ax=ax[i],pos=(1.05,0.5),s=(0.1,0.25))

    # plot spectra points
    Axy,_ = proj_pano(A.xyz, cam.pos, cam.ori, cam.fov, cam.dims, cam.step)
    Bxy,_ = proj_pano(B.xyz, cam.pos, cam.ori, cam.fov, cam.dims, cam.step)
    ax[i].scatter(Axy[:,0],Axy[:,1],c='white',edgecolors= "black")
    ax[i].scatter(Bxy[:,0],Bxy[:,1],c='white',edgecolors= "black")
    if i == 0:
        for px,py,t in zip(Axy[:,0],Axy[:,1],namesA):
            ax[i].text(px+15,py,t,va='center',bbox={'facecolor':'white', 'alpha':0.5})
        for px,py,t in zip(Bxy[:,0],Bxy[:,1],namesB):
            ax[i].text(px+15,py,t,va='center',bbox={'facecolor':'white', 'alpha':0.5})
(A+B).quick_plot(ax=ax[-1],hc=True,band_range=(2100., 2400.),pad=0.1)
ax[-1].set_xlabel("Wavelength (nm)")
ax[-1].set_title("d. Example spectra",loc='left')
fig.tight_layout()
fig.show()


#### Chapter 4: Detailed figures

Finally we create some specific figures highlighting different structures of interest.

In [ ]:
# helps keep memory usage down
%reset -f

In [ ]:
import hylite # if this doesn't work then please refer to Step 1
from hylite import io
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from hylite.analyse import colourise_mwl, plot_ternary
from hylite.project import Camera
from hylite.project import proj_pano
import matplotlib.gridspec as gridspec

In [ ]:
# figure settings for matplotlib
plt.rc('font', size=14)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=12)    # legend fontsize
plt.rc('figure', titlesize=20)  # fontsize of the figure title

# change background to black and text colour to white
mpl.rcParams['axes.facecolor'] = 'black'
mpl.rcParams["savefig.facecolor"] = 'black'
COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

In [ ]:
C = io.load('hyperclouds.hyc') # load hyperclouds HyCollection
M = io.load('mwl_maps.hdr') # load MWL features
O = hylite.HyCollection( 'visualisations','results/') # create output for storing mapping results

Maarmorilik Figure

In [ ]:
# RGB colour
fA = C.M1
fA.data = None # remove HSI data - we only want RGB
C.free()

# Iron index
fB = O.M1_Fe

# Carbonate MWL
fC,C_leg = colourise_mwl( M.M1_MWL.deepest(2290.,2380.),
                       strength=False, mode='p-d',
                       hue_map = (2300.,2350.), val_map=(0.0,0.2) )

In [ ]:
# create camera for plotting
pos = np.array([489430.,7888040.,50.])
ori = np.array([-100,50,-10])
dims = (1500,500)
fov=15
cam = Camera( pos, ori, 'pano', fov=fov, dims=dims, step=fov / dims[1])

In [ ]:
# render plot
fig,ax = plt.subplots(3,1, figsize=(20,16),facecolor='k')
t = ['a. RGB photogrammetry', 'b. Iron mineralogy',
     'c. Carbonate mineralogy' ]
for i,cld in enumerate([fA,fB,fC]):
    # render cloud
    render = cld.render(cam,'rgb')
    render.fill_holes()
    render.despeckle(3) # remove salt and pepper noise
    #render.data *= shade[...,None]**0.5 # apply hillshade

    # plot it
    ax[i].set_title(t[i],loc='left')
    render.quick_plot((0,1,2),vmin=2,vmax=98,ax=ax[i])

fig.tight_layout()

Black Angel Figure

In [ ]:
# create camera for plotting
pos = np.array([489430.,7888040.,200.])
ori = np.array([-95,35,-10])
dims = (600,400)
fov=12
cam = Camera( pos, ori, 'persp', fov=fov, dims=dims)

In [ ]:
# gather clouds
cA = O.M1_Fe
cB = O.M1_ENH
cC,legC = colourise_mwl( M.M1_MWL.deepest(2150.,2280.),
                       strength=False, mode='p-d',
                       hue_map = (2150.,2300.), val_map=(0.0,0.1) )
cD,legD = colourise_mwl( M.M1_MWL.deepest(2290.,2380.),
                       strength=False, mode='p-d',
                       hue_map = (2300.,2350.), val_map=(0.0,0.2) )

In [ ]:
# build spectral library
idx = [117507,  160292,  173009,  236904]
names = ['Calcitic marble', 'Dolomitic marble', 'Black Angel pelite', 'Graphitic marble']

# put results in a spectral library
lib = hylite.HyLibrary( C.M1.data[idx,:], lab=names, wav=C.M1.get_wavelengths() )
lib.decompress()

# extract positions also
lib.xyz = C.M1.xyz[idx,:]

# clean up to free memory
C.free()

In [ ]:
# compute hillshade factor
klm = cA.render(cam,'klm',fill_holes=True)
sundir = np.array([0.5,0.5,-1.0])
sundir = sundir / np.linalg.norm(sundir)
shade = np.clip( np.dot(-klm.data, sundir), 0.5, 1.0 )

In [ ]:
# setup figure
fig = plt.figure(figsize=(20,22),facecolor='k')
spec = gridspec.GridSpec(ncols=2, nrows=3, figure=fig)
ax = [
    fig.add_subplot(spec[0, :]),
    fig.add_subplot(spec[1, 0]),
    fig.add_subplot(spec[1, 1]),
    fig.add_subplot(spec[2, 0]),
    fig.add_subplot(spec[2, 1]),
]

# render spectra
ax[0].set_title("     a. Spectral lithology", loc='left')
lib.quick_plot(ax=ax[0],hc=True,pad=0.1, band_range=(2100., -1))
ax[0].set_yticks( ax[0].get_yticks() - 0.05 ) # move labels up
ax[0].set_yticklabels(names)
ax[0].tick_params(axis="y",direction="in", pad=-150) # move labels in

# render hypercloud
titles = ['b. Iron mineralogy', 'c. Saturation enhanced composite',
     'd. Silicate mineralogy', 'e. Carbonate mineralogy']
for i,cld in enumerate([cA,cB,cC,cD]):
    # render cloud
    render = cld.render(cam,'rgb')
    render.fill_holes()
    render.despeckle(3) # remove salt and pepper noise
    render.data *= shade[...,None]**0.5 # apply hillshade

    # plot it
    ax[i+1].set_title(titles[i],loc='left')
    render.quick_plot((0,1,2),vmin=2,vmax=98,ax=ax[i+1])

    # plot spectra points
    xy,_ = proj_pano(lib.xyz, cam.pos, cam.ori, cam.fov, cam.dims, cam.step)
    ax[i+1].scatter(xy[:,0],xy[:,1],c='white',edgecolors= "black")
    if i == 0:
        for px,py,t in zip(xy[:,0],xy[:,1],names):
            ax[i+1].text(px+15,py,t,va='center',bbox={'facecolor':'black', 'alpha':0.5})

# plot MWL legends
legC.plot( ax[3], pos=(0.3,-0.2), s=(0.5,0.15) )
legD.plot( ax[4], pos=(0.3,-0.2), s=(0.5,0.15) )
fig.tight_layout()